In [39]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import os
from os.path import join
from functions import analyse

In [40]:
x = tf.placeholder('float',[24,24,3])
y = tf.placeholder('float')


def pad(arr):
    r = np.zeros((24,24,3))
    r[:arr.shape[0],:arr.shape[1],:arr.shape[2]] = arr
    return r

def play_video(file = 'resources/test_video.mp4', width = 960, height = 540, flip = True):
    root = os.getcwd()
    ssd = cv.dnn.readNetFromCaffe(join(root, 'resources', 'MobileNetSSD_deploy.prototxt'),
                                  join(root, 'resources', 'MobileNetSSD_deploy.caffemodel'))
    frozen_tf_model = None
    threshold = 2000
    
    detector = cv.xfeatures2d.SURF_create(threshold)
    
    print("Starting Session")
    sess=tf.Session()

    tf.reset_default_graph()
    location = 'C:/Users/joear/OneDrive - Imperial College London/General/Code/Python/EDP/cnn_model/'
    imported_cnn = tf.train.import_meta_graph(location + 'current_model.meta')
    imported_cnn.restore(sess,tf.train.latest_checkpoint(location))
    print("Getting Graph")
    tf.get_default_graph()
    
    cap = cv.VideoCapture(file)
    ret, frame = cap.read()
    clone = cv.resize(frame, (width,height))
    if(flip): clone = cv.flip(clone, 0)
        
    print("Starting Loop")
    count = 0 ## To be deleted
    while ret:
        ret, frame = cap.read()
        if(flip): frame = cv.flip(frame, 0) 
        frame = cv.resize(frame, (1280, 720))
        clone = frame
        keypoints, detector, threshold, startX, endX = analyse(frame, ssd, frozen_tf_model, detector, 0,
                                                       threshold, use_classifier=False)
        
        frame = cv.drawKeypoints(frame, keypoints, None, (0, 255, 0), 4)
        cv.imshow("output", cv.resize(frame, (640, 360)))

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
        print("---------")
        if len(keypoints) > 0: 
            print("---------")
            for kp in keypoints:
                print('Coordinates: %d, %d' % (kp.pt[0], kp.pt[1]))
            
            kp_x = int(keypoints[0].pt[0])
            kp_y = int(keypoints[0].pt[1])
            print('First coordinate: %d, %d' % (kp_x, kp_y))
            roi = clone[kp_y-12:kp_y+12, kp_x-12:kp_x+12]
            roi_np = np.asarray(roi[:,:])
            roi_tf = tf.convert_to_tensor(roi_np)
            print("ROI   : ",type(roi))
            print("ROI NP: ",type(roi_np))
            print("ROI TF: ",type(roi_tf))
            print(roi_tf)
            cv.imshow("ROI", roi)
            cv.waitKey(0)
            cv.destroyWindow("ROI")

            result = sess.run({x: roi_tf})
            
            print(type(result))
            print(result)
            
            count += 1     ## To be deleted
            if count > 5:  ## To be deleted
                break      ## To be deleted
        
    cap.release()
    cv.destroyAllWindows()
    sess.close()

In [41]:
play_video()

Starting Session
INFO:tensorflow:Restoring parameters from C:/Users/joear/OneDrive - Imperial College London/General/Code/Python/EDP/cnn_model/current_model
Getting Graph
Starting Loop
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
Coordinates: 1085, 294
Coordinates: 867, 482
Coordinates: 1122, 690
Coordinates: 971, 482
Coordinates: 1097, 346
Coordinates: 1119, 644
Coordinates: 911, 482
Coordinates: 1115, 510
Coordinates: 1095, 429
Coordinates

ValueError: Fetch argument <tf.Tensor 'Const_1:0' shape=(24, 24, 3) dtype=uint8> cannot be interpreted as a Tensor. (Tensor Tensor("Const_1:0", shape=(24, 24, 3), dtype=uint8) is not an element of this graph.)

## Init values

x = tf.placeholder('float',[24,24,3])
y = tf.placeholder('float')

## Model Variables

#### 5 x 5, 1 input, produces 32 features
weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,3,32])),
           'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
           'W_fc':tf.Variable(tf.random_normal([6*6*64, 1024])),
           'out':tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
          'b_conv2':tf.Variable(tf.random_normal([64])),
          'b_fc':tf.Variable(tf.random_normal([1024])),
          'out':tf.Variable(tf.random_normal([n_classes]))}

x = tf.reshape(x, shape=[-1,24,24,3])

conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
conv1 = maxpool2d(conv1)

conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
conv2 = maxpool2d(conv2)

fc = tf.reshape(conv2,[-1,6*6*64])
fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])

fc = tf.nn.dropout(fc,keep_rate)

output = tf.matmul(fc, weights['out']) + biases['out']

